# Data wrangling

This notebook contains the procedures for building a data frame of county-level Census Bureau data over a period of years, and then combining this with CDC cardiovascular death rate data for these counties over the same range of years.

In [1]:
# Load libraries
import pandas as pd 
import requests
import numpy as np

## <u>Census data wrangling</u>

### Assembling Census API request components

<u>**Choosing which variables to request**</u>

Variables should be chosen that will be good predictive features for changes in cardiovascular death rate. Also, it would be best for these variables to correspond to things that can be addressed and influenced by policy changes or public awareness, and not things that lack any possibility of control. This is important since the desired result of this project is to provide a recommendation for preventing increases in the cardiovascular death rate. 

There are also a number of data sets provided through the Census API. The **American Community Survey (ACS) Comparison profiles** data set provides a way to compare values between years for a wide range of variables. Since the focus of this project is in the annual change in cardiovascular death rates, it would be useful to also have data for the annual changes in the values of the features. Using the **Comparison profiles** data set will allow for this to be achieved. 

From the documentation, there are 3 characteristic groups that could provide some good variables:

* Social - CP02
* Economic - CP03
* Housing - CP04

The social characteristics group contains variables of interest related to educational attainment. The values of these variables correspond to the population age 25 or older:

**Educacational Attainment**
* 058 - population age 25 or older
* 059 - less than 9th grade
* 060 - 9th-12th grade, no diploma
* 061 - HS graduate or equivalent
* 062 - some college, no degree
* 063 - Associate's degree
* 064 - Bachelor's degree
* 065 - Graduate or professional degree

In [2]:
# Create list of social characteristic variables
social = [
    '058','059','060','061',
    '062','063','064','065'
]

# Create dictionary for variable descriptions
social_dict = {
    '058':'edu_25_or_older',
    '059':'edu_lt9',
    '060':'edu_9_12',
    '061':'edu_hs_or_equiv',
    '062':'edu_col_no_degree',
    '063':'edu_assoc',
    '064':'edu_bach',
    '065':'edu_grad_or_prof'
}

The economic characteristics group contains variables of interest related to employment status, occupations and industries, commuting transportation methods, income and benefits, health insurance coverage, and poverty status.

**Employment Status**
* 009 - percent unemployed

**Commuting to work**
* 018 - workers 16 years and older
* 019 - drove alone
* 020 - carpooled
* 021 - public transport
* 022 - walked
* 023 - other means
* 024 - worked from home

**Occupations**
* 026 - civilian employed population 16 years and older
* 027 - management, business, science, and arts
* 028 - service
* 029 - sales and office
* 030 - natural resources, construction, and maintenance
* 031 - production, transportation, and material moving

**Industry**
* 032 - civilian employed population 16 years and older
* 033 - agriculture, forestry, fishing and hunting, and mining
* 034 - construction
* 035 - manufacturing
* 036 - wholesale trade
* 037 - retail and trade
* 038 - transportation and warehousing, and utilities
* 039 - information
* 040 - finance and insurance, and real estate and rental and leasing
* 041 - professional, scientific, and management, and administrative and waste management services
* 042 - educational services, and health care and social assistance
* 043 - arts, entertainment, and recreation, and accommodation and food services
* 044 - other services, except public administration
* 045 - public administration

**Income and Benefits**
* 051 - total households
* 062 - median household income (dollars)
* 063 - mean household income (dollars)
* 064 - households with earnings
* 065 - mean household earnings income (dollars)
* 066 - households with Social Security
* 067 - mean household Social Security income (dollars)
* 068 - households with retirement income
* 069 - mean household retirement income (dollars)
* 070 - households with Supplemental Security income
* 071 - mean household Supplemental Security income (dollars)
* 072 - households with cash public assistance income
* 073 - mean household cash public assistance income (dollars)
* 074 - households with Food Stamp/SNAP benefits in past 12 months

**Health Insurance Coverage**
* 095 - civilian noninstitutionalized population
* 096 - with health insurance coverage
* 097 - with private health insurance
* 098 - with public coverage
* 099 - no health insurance coverage

**Poverty Status**
* 119 - percent of families at or below poverty level
* 128 - percent of all people at or below poverty level 

In [3]:
# Create list of economic characteristic variables
economic = [
    '009','018','019','020',
    '021','022','023','024',
    '026','027','028','029',
    '030','031','032','033',
    '034','035','036','037',
    '038','039','040','041',
    '042','043','044','045',
    '051','062','063','064',
    '065','066','067','068',
    '069','070','071','072',
    '073','074','095','096',
    '097','098','099','119',
    '128'
]

# Create dictionary for variable descriptions
economic_dict = {
    '009':'emp_pct_unemplyd','018':'comm_16_or_older',
    '019':'comm_drove_alone','020':'comm_carpooled',
    '021':'comm_pub_trans','022':'comm_walk',
    '023':'comm_other','024':'comm_home',
    '026':'occ_16_or_older','027':'occ_mgmt_bus_sci_art',
    '028':'occ_service','029':'occ_sls_off',
    '030':'occ_natrsrc_constr_mntnc','031':'occ_prdctn_trans_mtrlmvng',
    '032':'ind_16_or_older','033':'ind_agg_fstry_fsh_hnt_mng',
    '034':'ind_constr','035':'ind_mnfctrng','036':'ind_trade_whlsl',
    '037':'ind_trade_retail','038':'ind_trans_wrhsng_util',
    '039':'ind_info','040':'ind_fnce_inrs_realestate',
    '041':'ind_prof_sci_mgmt_admn_wstmgmt','042':'ind_edu_hlthcare_socialasst',
    '043':'ind_arts_entrtnmt_rec_accmdtn_food','044':'ind_other_notpubadmn',
    '045':'ind_pubadmn','051':'inc_ttl_hshlds',
    '062':'inc_med_hshld','063':'inc_mean_hshld',
    '064':'inc_pct_hshlds_earnings','065':'inc_mean_earnings',
    '066':'inc_pct_hshlds_sclsec','067':'inc_mean_sclsec',
    '068':'inc_pct_hshlds_retrmnt','069':'inc_mean_retrmnt',
    '070':'inc_pct_hshlds_suppsec','071':'inc_mean_suppsec',
    '072':'inc_pct_hshlds_pubasst','073':'inc_mean_pubasst',
    '074':'inc_pct_hshlds_stamps_snap','095':'insr_civ_noninstnlz_pop',
    '096':'insr_with_insr','097':'insr_with_insr_prvt',
    '098':'insr_with_inr_pub','099':'insr_no_insr',
    '119':'pov_pct_fmly_at_or_blw','128':'pov_pct_ppl_at_or_blw'
}

The housing characteristic group contains variables related to monthly owner cost and rent costs.

*Starting in 2015, all of these housing variable codes were increased by 2*

**Selected monthly owner costs as a percentage of household income (SMOCAPI)**

**Variables for housing units with a mortage**
* 108 - housing units with a mortage
* 109 - SMOCAPI less than 20 %
* 110 - SMOCAPI 20 - 24.9 %
* 111 - SMOCAPI 25 - 29.9 %
* 112 - SMOCAPI 30 - 34.9 % 
* 113 - SMOCAPI 35 % or more
* 114 - not computed 

**Variables for housing units without a mortage**
* 115 - housing units without a mortage
* 116 - SMOCAPI less than 10 %
* 117 - SMOCAPI 10 - 14.9 %
* 118 - SMOCAPI 15 - 19.9 % 
* 119 - SMOCAPI 20 - 24.9 %
* 120 - SMOCAPI 25 - 29.9 %
* 121 - SMOCAPI 30 - 34.9 %
* 122 - SMOCAPI 35 % or more
* 123 - not computed

**Gross rent**
* 124 - occupied units paying rent
* 132 - median gross rent (dollars)
* 133 - no rent paid

**Gross rent as a percentage of household income (GRAPI)**
* 134 - occupied units paying rent
* 135 - GRAPI less than 15 %
* 136 - GRAPI 15 - 19.9 %
* 137 - GRAPI 20 - 24.9 %
* 138 - GRAPI 25 - 29.9 %
* 139 - GRAPI 30 - 34.9 %
* 140 - GRAPI 35 % or more
* 141 - not computed 

In [4]:
# Create a list of housing characteristic variables
housing = [
    '108','109','110','111',
    '112','113','114','115',
    '116','117','118','119',
    '120','121','122','123',
    '124','132','133','134',
    '135','136','137','138',
    '139','140','141'
]

# Create dictionary for variable descriptions
housing_dict = {
    '108':'smocapi_mrtg_units','109':'smocapi_mrtg_under_20%',
    '110':'smocapi_mrtg_20_25%','111':'smocapi_mrtg_25_30%',
    '112':'smocapi_mrtg_30_35%','113':'smocapi_mrtg_35%_or_more',
    '114':'smocapi_mrtg_not_computed','115':'smocapi_nomrtg_units',
    '116':'smocapi_nomrtg_under_10%','117':'smocapi_nomrtg_10_15%',
    '118':'smocapi_nomrtg_15_20%','119':'smocapi_nomrtg_20_25%',
    '120':'smocapi_nomrtg_25_30%','121':'smocapi_nomrtg_30_35%',
    '122':'smocapi_nomrtg_35%_or_more','123':'smocapi_nomrtg_not_computed',
    '124':'rent_occ_units','132':'rent_med_gross_rent',
    '133':'rent_not_paid','134':'grapi_units','135':'grapi_under_15%',
    '136':'grapi_15_20%','137':'grapi_20_25%','138':'grapi_25_30%',
    '139':'grapi_30_35%','140':'grapi_35%_or_more','141':'grapi_not_computed'
}

Each of these lists contains the codes for the variables we are interested in, but there are a few more steps that must be completed before we can begin making the API requests.

We want data over the range of years 2011-2018 for every county in the United States. For each data point we want the value of each of these variables for that year, but we also want the value from the previous year, which is why we chose the **Comparison profiles** dataset.

This data set also has the advantage of providing the statistical significance in the change of the variables between years. 

To summarize; 

**We want, for each of these variables, the value of the currently requested year, the value of the previous year, their difference, and the statistical significance of their difference**

Looking at the documentation, we see that each list must be used to create a few sets of variables with distinct formats.

**For current year data, the variables take the form**
* CP0X_YYYY_ijkE 

Where $X \in \{2,3,4\}$, *YYYY* is the current year, and *ijk* is the variable code. For the value of the previous year, the value for *YYYY* is simply reduced by 1.

**For statistical signficance in the difference between years, use the following format**
* CP0X_YYYYtoZZZZ_ijkSS

Where again $X \in \{2,3,4\}$, *YYYY* is the current year, *ijk* is the variable code, and finally *ZZZZ* is the previous year.

In [5]:
# Function to create request variable names. 
def get_req_vars(codes, group,year):
    """
    This function takes as input a list of Census 
    variable codes belonging to a specified group,
    along with a particular year. It then returns 
    a list of variable names in the format needed
    for the Census API requests.
    """
    # Initialize list to store variable names
    req_vars = []
    
    # Iterate over the list of codes
    for code in codes:
        # Declare the code suffixes
        est_sfx = code+'E'
        ss_sfx = code+'SS'
        # Create the names of estimates
        est_curr = '_'.join([group,str(year),est_sfx])
        est_prev = '_'.join([group,str(year-1),est_sfx]) 
        # Create the name for the statistical significance
        inner = str(year)+'to'+str(year-1)
        diff_ss = '_'.join([group,inner,ss_sfx])
        # Append the names to the request variables list
        req_vars.append(est_curr)
        req_vars.append(est_prev)
        req_vars.append(diff_ss)
    
    # Return the list of request variable names
    return req_vars    

# Function to build request predicate
def get_preds(req_vars,county='*',state='*'):
    """
    This function builds the predicate component
    for the Census API request. It takes as input
    a list of variable names, and a specified county
    and state. By default, all states and counties are 
    selected.
    """
    # Initialize predicates dictionary
    preds = {}
    # Create the key, value pairs
    preds['get'] = ','.join(req_vars)
    preds['for'] = 'county:{}'.format(county)
    preds['in'] = 'state:{}'.format(state)
    # Return predicates dictionary
    return preds

We are limited to requesting 50 variables at a time. To resolve this issue, we could try individually requesting the variables of each group, and then merging their resultant data frames.

However, even if we split into separate groups, the economic and housing groups will still contain too many variables. So we need to further split these groups into smaller components. Reducing each code list to have maximum length of 15 should resolve the issue.

As previously mentioned, the housing variable codes all increase by 2 starting in 2015. The request functions must be written to account for this, otherwise the wrong variables will be requested starting in 2015.

In [6]:
# Function to create DataFrame from group variable codes
def get_group_df(codes,group,year,county='*',state='*'):
    """
    This function takes as input a list of variable codes for
    a particular group of the comparison profile data set of 
    the Census API, as well as a specified year, county code, 
    and state code. It outputs a DataFrame made from the response
    object of an API request.
    
    Conditional logic is used to handle group CP04 for years 2015
    and later.
    """
    # Define base url included in all the requests.
    HOST = "https://api.census.gov/data"
    # Specify the comparison profile dataset.
    dataset = "acs/acs1/cprofile"
    # Create the request url.
    url  = '/'.join([HOST,str(year),dataset])
    
    # Get the list of request variable names.
    req_vars = ['NAME']+get_req_vars(codes,group=group,year=year)
    
    # Get columns labels.
    cols = req_vars+['state','county']
    
    # Check for housing group 2015 or later
    if (group=='CP04')&(year>=2015):
        # Increase housing codes by 2
        new_codes = [str(int(code)+2) for code in codes]
        # Remake the request variables.
        req_vars = ['NAME']+get_req_vars(new_codes,group=group,year=year)
        
    
    # Create the predicates dictionary.
    preds = get_preds(req_vars,county=county,state=state)
    
    # Get the response object.
    resp = requests.get(url,params=preds).json()
    
    # Store response in DataFrame and return it.
    df = pd.DataFrame(resp[1:],columns=cols)
    return df

In [7]:
# Function to get DataFrame from split code lists
def get_group_df_split(codes_split,group,year,county='*',state='*'):
    """
    Function takes as input a list of lists containing the 
    variable codes for a group, and returns a merged DataFrame
    from the request of each individual split code list.
    """
    # Get DataFrame from first split
    df = get_group_df(codes_split[0],group=group,
                      year=year,county=county,state=state)
    
    # Merge df with successive split DataFrames
    for split in codes_split[1:]:
        df_split = get_group_df(split,group=group,year=year,
                                county=county,state=state)
        df = pd.merge(df,df_split,on=['NAME','state','county'])
        
    # Return the merged DataFrame
    return df

In [8]:
# Function to merge group DataFrames 
def merge_group_dfs(df_list,year):
    """
    Function takes a list of DataFrames as input,
    merges them on matching column keys, reorders the
    columns, and adds an additional column to record the
    year.
    """
    # Store the first DataFrame
    df = df_list[0]
    
    # Merge df with remaining DataFrames
    for df_ in df_list[1:]:
        df = pd.merge(df,df_,on=['NAME','state','county'])
    
    # Create a column to record year value
    df['year'] = year
    
    # Rearrange the column order
    cols = df.columns.to_list()
    id_cols = ['NAME','county','state','year']
    for col in id_cols:
        cols.remove(col)
    cols = id_cols+cols
    df = df[cols]
    current = '_'+str(year)+'_'
    previous = '_'+str(year-1)+'_'
    compare = '_'+str(year)+'to'+str(year-1)+'_'
    cols = [col.replace(current,'_current_') for col in cols]
    cols = [col.replace(previous,'_previous_') for col in cols]
    cols = [col.replace(compare,'_') for col in cols]
    df.columns = cols
    
    # Return the final DataFrame
    return df

In [9]:
# Define the range of years to request
years = np.arange(2011,2019)

# Initialize list to store each year
df_years = []

# Split economic codes into 4 lists
economic1 = economic[:15]
economic2 = economic[15:30]
economic3 = economic[30:45]
economic4 = economic[45:]

# Split housing codes into 2 lists
housing1 = housing[:15]
housing2 = housing[15:]

# Define the split code lists
e_split = [economic1,economic2,economic3,economic4]
h_split = [housing1,housing2]

# Iterate over years
for year in years:
    # Get the group DataFrames
    s_df = get_group_df(social,group='CP02',year=year)
    e_df = get_group_df_split(e_split,group='CP03',year=year)
    h_df = get_group_df_split(h_split,group='CP04',year=year)
    
    # Store group DataFrames in list
    df_list = [s_df,e_df,h_df]
    
    # Merge the DataFrames 
    df = merge_group_dfs(df_list,year)
    
    # Append year DataFrame to list
    df_years.append(df)

# Concatenate the DataFrames
census = pd.concat(df_years,ignore_index=True)
census['county'] = census['county'].astype(int)
census['state'] = census['state'].astype(int)
census['year'] = census['year'].astype(int)
census.head()

,NAME,county,state,year,CP02_current_058E,CP02_previous_058E,CP02_058SS,CP02_current_059E,CP02_previous_059E,CP02_059SS,...,CP04_138SS,CP04_current_139E,CP04_previous_139E,CP04_139SS,CP04_current_140E,CP04_previous_140E,CP04_140SS,CP04_current_141E,CP04_previous_141E,CP04_141SS
0,"Calhoun County, Michigan",25,26,2011,90930,90844,None,3.0,4.6,*,...,None,10.4,10.5,None,47.4,38.2,None,1126,819,None
1,"Clinton County, Michigan",37,26,2011,49654,48361,*,2.5,2.0,None,...,*,13.4,12.7,None,32.8,45.7,None,855,668,None
2,"Eaton County, Michigan",45,26,2011,74278,73273,*,1.2,2.6,None,...,*,9.6,11.9,None,39.2,40.9,None,790,417,None
3,"Genesee County, Michigan",49,26,2011,280301,280659,None,2.7,2.7,None,...,None,9.4,8.3,None,52.7,55.0,None,4706,4204,None
4,"Grand Traverse County, Michigan",55,26,2011,62370,61645,None,1.9,1.3,None,...,None,10.0,6.0,None,47.7,50.4,None,151,706,*


Now we need to combine this with the cardiovascular death rate DataFrame.

## <u>CDC data wrangling</u>

A file containing county-level cardiovascular deaths and death rate data was obtained using the **CDC WONDER** interface. This file must now be loaded into a DataFrame and cleaned so that it can be combined with the DataFrame constructed from the Census API requests.

In [10]:
# Define strings that denote null values.
na_values = ['Suppressed','Unreliable','Missing']

# Read in tab-delimited text file.
df = pd.read_table('data/cdc_data.txt',na_values=na_values)

# Drop rows containing values for Notes.
df = df[df.Notes.isnull()].drop('Notes',axis=1)

# Drop rows with null entries.
df = df.dropna().reset_index(drop=True)

# Remove state code from county codes and cast to int.
df['County Code'] = df['County Code'].astype(int).astype(str)
df['County Code'] = df['County Code'].apply(lambda x: int(x[-3:]))


# Change columns to datatype int.
df['State Code'] = df['State Code'].astype(int)
df['Year'] = df['Year'].astype(int)
df['Crude Rate'] = df['Crude Rate'].astype(float)

# Reformat column labels.
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

# Define reordered subset of columns.
cols = [
    'county_code','state_code',
    'year','deaths','population',
    'crude_rate'
]

# Select subset of columns.
df = df[cols]

# Create a new column recording the change in CDR from the previous year.
df['rate_change'] = np.nan

# Iterate over rows and assign value for rate_change.
for index, row in df.iterrows():
    # If year is 2010, continue.
    if row.year==2010:
        continue
    # Get sample identifiers.
    rate = row.crude_rate
    county_ = row.county_code
    state_ = row.state_code
    # Get all previous entries for county.
    df_pre = df.iloc[:index].copy()
    df_pre = df_pre[(df_pre.county_code==county_)
                    &(df_pre.state_code==state_)]
    # If no previous entries, then continue.
    if len(df_pre)==0:
        continue
    # Assign 1 if rate has increased, 0 otherwise.
    rate_pre = df_pre.iloc[-1].crude_rate
    if rate <= rate_pre:
        df.loc[index,'rate_change'] = 0
    else:
        df.loc[index,'rate_change'] = 1
        
# Drop all null values.
cdc = df.dropna().reset_index(drop=True)


# Rename cdc columns to merge with census
cdc_cols = [
    'county','state',
    'year','deaths',
    'population','crude_rate',
    'rate_change'
]
cdc.columns = cdc_cols
cdc.head()

,county,state,year,deaths,population,crude_rate,rate_change
0,1,1,2011,160.0,55267.0,289.5,1.0
1,1,1,2012,161.0,55514.0,290.0,1.0
2,1,1,2013,174.0,55246.0,315.0,1.0
3,1,1,2014,153.0,55395.0,276.2,0.0
4,1,1,2015,150.0,55347.0,271.0,0.0


In [11]:
# Merge the census and cdc DataFrames
df = pd.merge(census,cdc,on=['county','state','year'])
df.head()

,NAME,county,state,year,CP02_current_058E,CP02_previous_058E,CP02_058SS,CP02_current_059E,CP02_previous_059E,CP02_059SS,...,CP04_current_140E,CP04_previous_140E,CP04_140SS,CP04_current_141E,CP04_previous_141E,CP04_141SS,deaths,population,crude_rate,rate_change
0,"Calhoun County, Michigan",25,26,2011,90930,90844,None,3.0,4.6,*,...,47.4,38.2,None,1126,819,None,441.0,135490.0,325.5,0.0
1,"Clinton County, Michigan",37,26,2011,49654,48361,*,2.5,2.0,None,...,32.8,45.7,None,855,668,None,179.0,75469.0,237.2,0.0
2,"Eaton County, Michigan",45,26,2011,74278,73273,*,1.2,2.6,None,...,39.2,40.9,None,790,417,None,291.0,108056.0,269.3,1.0
3,"Genesee County, Michigan",49,26,2011,280301,280659,None,2.7,2.7,None,...,52.7,55.0,None,4706,4204,None,1400.0,422080.0,331.7,1.0
4,"Grand Traverse County, Michigan",55,26,2011,62370,61645,None,1.9,1.3,None,...,47.7,50.4,None,151,706,*,229.0,88349.0,259.2,1.0


Now we need to get the difference between the current and previous year for each census variable and record this in a new column.

In [12]:
# Reorder the colums
cols = df.columns.to_list()
cols = cols[:4]+cols[-4:]+cols[4:-4]
df = df[cols]
df.head()

,NAME,county,state,year,deaths,population,crude_rate,rate_change,CP02_current_058E,CP02_previous_058E,...,CP04_138SS,CP04_current_139E,CP04_previous_139E,CP04_139SS,CP04_current_140E,CP04_previous_140E,CP04_140SS,CP04_current_141E,CP04_previous_141E,CP04_141SS
0,"Calhoun County, Michigan",25,26,2011,441.0,135490.0,325.5,0.0,90930,90844,...,None,10.4,10.5,None,47.4,38.2,None,1126,819,None
1,"Clinton County, Michigan",37,26,2011,179.0,75469.0,237.2,0.0,49654,48361,...,*,13.4,12.7,None,32.8,45.7,None,855,668,None
2,"Eaton County, Michigan",45,26,2011,291.0,108056.0,269.3,1.0,74278,73273,...,*,9.6,11.9,None,39.2,40.9,None,790,417,None
3,"Genesee County, Michigan",49,26,2011,1400.0,422080.0,331.7,1.0,280301,280659,...,None,9.4,8.3,None,52.7,55.0,None,4706,4204,None
4,"Grand Traverse County, Michigan",55,26,2011,229.0,88349.0,259.2,1.0,62370,61645,...,None,10.0,6.0,None,47.7,50.4,None,151,706,*


In [13]:
# Initialize list for reordered columns
diff_cols = []

# Get differences columns of each group
for code in social:
    # Get the column labels
    c = 'CP02_current_'+code+'E'
    p = 'CP02_previous_'+code+'E'
    d = 'CP02_diff_'+code+'E'
    s = 'CP02_'+code+'SS'
    diff_cols+=[c,p,d,s]
    
    # Recast the value columns as floats
    df[c] = df[c].astype(float)
    df[p] = df[p].astype(float)
    
    # Replace negative values with nan
    df[c] = df[c].where(df[c]>=0)
    df[p] = df[p].where(df[p]>=0)
    
    df[s] = df[s].astype(str)
    df[s] = df[s].map({'nan':0,'*':1})
    
    # Create the difference column
    df[d] = df[c] - df[p]
    
for code in economic:
    # Get the column labels
    c = 'CP03_current_'+code+'E'
    p = 'CP03_previous_'+code+'E'
    d = 'CP03_diff_'+code+'E'
    s = 'CP03_'+code+'SS'
    diff_cols+=[c,p,d,s]
    
    # Recast the value columns as floats
    df[c] = df[c].astype(float)
    df[p] = df[p].astype(float)
    
    # Replace negative values with nan
    df[c] = df[c].where(df[c]>=0)
    df[p] = df[p].where(df[p]>=0)
    
    df[s] = df[s].astype(str)
    df[s] = df[s].map({'nan':0,'*':1})
    
    # Create the difference column
    df[d] = df[c] - df[p]
    
for code in housing:
    # Get the column labels
    c = 'CP04_current_'+code+'E'
    p = 'CP04_previous_'+code+'E'
    d = 'CP04_diff_'+code+'E'
    s = 'CP04_'+code+'SS'
    diff_cols+=[c,p,d,s]
    
    # Recast the value columns as floats
    df[c] = df[c].astype(float)
    df[p] = df[p].astype(float)
    
    # Replace negative values with nan
    df[c] = df[c].where(df[c]>=0)
    df[p] = df[p].where(df[p]>=0)
    
    df[s] = df[s].astype(str)
    df[s] = df[s].map({'nan':0,'*':1})
    
    # Create the difference column
    df[d] = df[c] - df[p]

# Reorder the columns
cols = cols[:8]+diff_cols
df = df[cols]
df.head()

,NAME,county,state,year,deaths,population,crude_rate,rate_change,CP02_current_058E,CP02_previous_058E,...,CP04_diff_139E,CP04_139SS,CP04_current_140E,CP04_previous_140E,CP04_diff_140E,CP04_140SS,CP04_current_141E,CP04_previous_141E,CP04_diff_141E,CP04_141SS
0,"Calhoun County, Michigan",25,26,2011,441.0,135490.0,325.5,0.0,90930.0,90844.0,...,-0.1,0,47.4,38.2,9.2,0,1126.0,819.0,307.0,0
1,"Clinton County, Michigan",37,26,2011,179.0,75469.0,237.2,0.0,49654.0,48361.0,...,0.7,0,32.8,45.7,-12.9,0,855.0,668.0,187.0,0
2,"Eaton County, Michigan",45,26,2011,291.0,108056.0,269.3,1.0,74278.0,73273.0,...,-2.3,0,39.2,40.9,-1.7,0,790.0,417.0,373.0,0
3,"Genesee County, Michigan",49,26,2011,1400.0,422080.0,331.7,1.0,280301.0,280659.0,...,1.1,0,52.7,55.0,-2.3,0,4706.0,4204.0,502.0,0
4,"Grand Traverse County, Michigan",55,26,2011,229.0,88349.0,259.2,1.0,62370.0,61645.0,...,4.0,0,47.7,50.4,-2.7,0,151.0,706.0,-555.0,1


In [14]:
# Write the DataFrame to csv file
df.to_csv('data/cdr_data.csv')